In [1]:
from __future__ import division, print_function, absolute_import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
ethnea_df = pd.read_csv('names_ethnea_genni_country.csv')
#ethnea_df

In [3]:
# First thing first, use the character feature for making the dnn model
def extract_structure(word,n_char=2):
    x_struct = []
    word_len = len(word) + n_char
    n_char-=1
    counter = 0
    for i in range(word_len):
        end = i+1
        start = (i - n_char) if (i - n_char) > 0 else 0
        if word[start:end]!='_' and word[start:end]!='':
        #if word[start:end]!='_':
            x_struct.append(word[start:end])
    return x_struct

first_name_struct = ethnea_df.First.apply(lambda x: extract_structure(x.lower(),2))
last_name_struct = ethnea_df.Last.apply(lambda x: extract_structure(x.lower(),2))                                                                

In [4]:
# make struct dictionary
struct_dict = {}
for name_struct_i in first_name_struct:
    for struct_j in name_struct_i:
        if struct_j not in struct_dict:
            struct_dict[struct_j]=0
        struct_dict[struct_j]+=1
for name_struct_i in last_name_struct:
    for struct_j in name_struct_i:
        if struct_j not in struct_dict:
            struct_dict[struct_j]=0
        struct_dict[struct_j]+=1

In [5]:
struct_dict_keys = list(struct_dict.keys())
ethnic_series = ethnea_df['Ethnea'].str.lower()
pub_series = ethnea_df.PubCountry.str.lower()
ethnic_keys = list(np.unique(ethnic_series.values))
pub_keys = list(np.unique(pub_series.values))

In [12]:
len(pub_keys)

275

In [6]:
# load test train data
with open('train_test_full_index.pickle', 'rb') as f:
    trainIndex,testIndex,trainY,testY = pickle.load(f)

In [7]:
# transform the dataset into structure
def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

In [31]:
def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

#data_source = full_name_struct.apply(lambda x: transform_structure(x))

def generate_batch(first_name, last_name, i, batch_size=10000):
    len_name = len(first_name)
    print(len_name)
    start = i*batch_size
    print(start)
    end = start+batch_size if start+batch_size < len_name else len_name
    len_mat = end - start
    #first_name_ds_mat = np.zeros((len_mat,50),dtype=np.int32)
    #last_name_ds_mat = np.zeros((len_mat,50),dtype=np.int32)
    first_name_ds_mat = first_name[start:end]
    last_name_ds_mat = last_name[start:end]
    first_name_ds_mat = pad_sequences(first_name_ds_mat.apply(lambda x: transform_structure(x)),maxlen=50,value=0.)
    first_name_ds_mat = first_name_ds_mat.reshape(first_name_ds_mat.shape[0],1,first_name_ds_mat.shape[1])
    last_name_ds_mat = pad_sequences(last_name_ds_mat.apply(lambda x:transform_structure(x)),maxlen=50,value=0.)
    last_name_ds_mat = last_name_ds_mat.reshape(last_name_ds_mat.shape[0],1,last_name_ds_mat.shape[1])

    """
    for i in range(len_mat):
        x = first_name_struct.iloc[i]
        for y in x:
            first_name_ds_mat[i,struct_dict_keys.index(y)]+=1
        x = last_name_struct.iloc[i]
        for y in x:
            last_name_ds_mat[i,struct_dict_keys.index(y)]+=1
    """
    return first_name_ds_mat,last_name_ds_mat, range(start,end)

In [9]:
def transform_labels(x,my_keys):
    y = np.zeros(len(my_keys))
    y[my_keys.index(x)]=1
    return y


labels = np.array(list(map(lambda x: transform_labels(x,ethnic_keys),ethnic_series)))
#pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys),pub_series)))

In [10]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tflearn.data_utils import to_categorical, pad_sequences



Using TensorFlow backend.


In [11]:
?Input

Object `Input` not found.


In [12]:
from keras.layers import Input
from keras.models import Model

"""
model = Sequential()
model.add(Embedding(len(name_struct_keys)+1,embedding_vector_length,input_length=max_sequence))
model.add(Conv1D(filters=embedding_vector_length,kernel_size=3,padding='same',activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(LSTM(lstm_layer,dropout=0.8))
model.add(Bidirectional(LSTM(max_sequence*2,return_sequences=False),input_shape=(max_sequence,1)))
#model.add(TimeDistributed(keras.layers.Dense(len(ethnic_keys),activation='softmax')))
model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
#model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
"""

# max sequence length
seq_length = 100
# multi input with single output

# first name input
first_name_input = Input(shape=(1,50),name='first_name_input')
last_name_input = Input(shape=(1,50),name='last_name_input')

# pub input
pub_input = Input(shape=(len(pub_keys),),name='pub_input')

# first tensor for first name
first_name_l = Bidirectional(LSTM(1000,return_sequences=False))(first_name_input)
last_name_l = Bidirectional(LSTM(1000,return_sequences=False))(last_name_input)

# merge the two layer together
merge_first_last = keras.layers.concatenate([first_name_l,last_name_l])

# stack dense network for memory
full_conn_merge = Dense(1000, activation='relu')(merge_first_last)
x = keras.layers.concatenate([full_conn_merge,pub_input])
x = Dense(500, activation='relu')(x)
output_l = Dense(len(ethnic_keys),activation='softmax')(x)

model = Model(inputs=[first_name_input, last_name_input,pub_input], outputs=[output_l])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
first_name_input (InputLayer)    (None, 1, 50)         0                                            
____________________________________________________________________________________________________
last_name_input (InputLayer)     (None, 1, 50)         0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 2000)          8408000     first_name_input[0][0]           
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 2000)          8408000     last_name_input[0][0]            
___________________________________________________________________________________________

In [ ]:
model.summary()
first_trainX = first_name_struct[trainIndex]
first_testX = first_name_struct[testIndex]
last_trainX = last_name_struct[trainIndex]
last_testX = last_name_struct[testIndex]
trainY = labels[trainIndex]
testY = labels[testIndex]

#trainX =np.array([to_categorical(x,nb_classes=len(struct_dict_keys)+1) for x in trainX])
#testX =np.array([to_categorical(x,nb_classes=len(struct_dict_keys)+1) for x in testX])

mini_batch_size = 20000
len_mini_batch = round(len(trainY)/mini_batch_size)
batch_size = 5000

for x in range(1):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys),pub_series[trainIndex].iloc[batch_range])))
        model.fit([y_first_trainX, y_last_trainX, pub_country],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
first_name_input (InputLayer)    (None, 1, 50)         0                                            
____________________________________________________________________________________________________
last_name_input (InputLayer)     (None, 1, 50)         0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 2000)          8408000     first_name_input[0][0]           
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 2000)          8408000     last_name_input[0][0]            
___________________________________________________________________________________________

20000/20000 [==============================] - 41s - loss: 0.7723 - acc: 0.7765     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.7678 - acc: 0.7769     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.7681 - acc: 0.7791     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.7512 - acc: 0.7848     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.7639 - acc: 0.7778     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.7570 - acc: 0.7809     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.7708 - acc: 0.7730     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.7668 - acc: 0.7761     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.7754 - acc: 0.7750     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.7734 - acc: 0.7743     
Epoch 1/1
20000/20000 [==============================] - 42s

20000/20000 [==============================] - 41s - loss: 0.6490 - acc: 0.8124     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6463 - acc: 0.8117     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.6535 - acc: 0.8108     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6348 - acc: 0.8160     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6517 - acc: 0.8105     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6406 - acc: 0.8106     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.6489 - acc: 0.8110     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6393 - acc: 0.8162     


In [20]:
batch_range
#pub_series[trainIndex][list(batch_range)]

range(3520000, 3540000)

In [56]:
def evaluate(firstX, lastX, testY, testIndex, mini_batch_size=20000):
    test_len = len(testY)
    len_mini_batch = round(test_len/mini_batch_size)
    scores = np.zeros(len_mini_batch)
    for i in range(len_mini_batch):
        y_first_trainX,y_last_trainX,batch_range = generate_batch(firstX,lastX,i,mini_batch_size)
        pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys), pub_series[testIndex].iloc[batch_range])))
        scores[i] = model.evaluate([y_first_trainX,y_last_trainX,pub_country],testY[batch_range])[1]
        print(scores[i])
    return np.average(scores)
    #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)

In [ ]:
evaluate(first_testX,last_testX, testY, testIndex)
#test
#y_first_trainX,y_last_trainX,batch_range = generate_batch(first_testX,last_testX,1,mini_batch_size)

886817
0
20000/20000 [==============================] - 30s    
0.81265
886817
20000
20000/20000 [==============================] - 30s    
0.81075
886817
40000
 3072/20000 [===>..........................] - ETA: 25s

In [50]:
#pub_series[testIndex].iloc[batch_range]
pub_series[testIndex].iloc[20000]

'china'

In [44]:
mini_batch_size = 50000
len_mini_batch = round(len(trainY)/mini_batch_size)
batch_size = 5000

for x in range(10):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        model.fit([y_first_trainX, y_last_trainX],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3415 - acc: 0.3281    
Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3498 - acc: 0.3248    
Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3490 - acc: 0.3256    
Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3557 - acc: 0.3225    
Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3430 - acc: 0.3260    
Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3428 - acc: 0.3277    
Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3461 - acc: 0.3250    
Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3501 - acc: 0.3234    
Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3491 - acc: 0.3231    
Epoch 1/1
50000/50000 [==============================] - 14s - loss: 2.3422 - acc: 0.3267    
Epoch 1/1
50000/50000 [==============================] - 14s

50000/50000 [==============================] - 14s - loss: 2.3462 - acc: 0.3253    
Epoch 1/1
50000/50000 [==============================] - 32s - loss: 2.3493 - acc: 0.3242    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2.3398 - acc: 0.3256    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2.3365 - acc: 0.3241    
Epoch 1/1
50000/50000 [==============================] - 45s - loss: 2.3405 - acc: 0.3279    
Epoch 1/1
50000/50000 [==============================] - 48s - loss: 2.3457 - acc: 0.3266    
Epoch 1/1
50000/50000 [==============================] - 70s - loss: 2.3442 - acc: 0.3263    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2.3426 - acc: 0.3247    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2.3412 - acc: 0.3263    
Epoch 1/1
50000/50000 [==============================] - 51s - loss: 2.3469 - acc: 0.3249    
Epoch 1/1
50000/50000 [==============================] - 65s - loss: 2

50000/50000 [==============================] - 66s - loss: 2.3469 - acc: 0.3219    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2.3504 - acc: 0.3225    
Epoch 1/1
50000/50000 [==============================] - 36s - loss: 2.3459 - acc: 0.3239    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2.3405 - acc: 0.3265    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2.3475 - acc: 0.3233    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2.3372 - acc: 0.3297    
Epoch 1/1
47268/47268 [==============================] - 63s - loss: 2.3396 - acc: 0.3272    
Epoch 1/1
50000/50000 [==============================] - 65s - loss: 2.3407 - acc: 0.3281    
Epoch 1/1
50000/50000 [==============================] - 65s - loss: 2.3495 - acc: 0.3248    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2.3485 - acc: 0.3256    
Epoch 1/1
50000/50000 [==============================] - 66s - loss: 2

KeyboardInterrupt: 

In [28]:
y_first_trainX.reshape(y_first_trainX.shape[0],1,y_first_trainX.shape[1]).shape

(10000, 1, 50)

In [39]:
def trans_name(name):
    name = name.lower()
    # transform space into underscore
    name = '_'+name.replace(' ','_')+'_'
    #transform the name into sequence structure
    ext_name = extract_structure(name)
    trans_name = transform_structure(ext_name)
    #name_ds_mat = np.zeros((1,len(struct_dict_keys)),dtype=np.int32)
    #for i,x in enumerate(trans_name):
    #    name_ds_mat[0,x-1]+=1    
    trans_name = pad_sequences([trans_name], maxlen=50,value=0.)
    trans_name = trans_name.reshape(trans_name.shape[0],1,trans_name.shape[1])

    return trans_name

def predict_ethnicity(fname,lname):
    # lower case the name
    fnamex = trans_name(fname)
    lnamex = trans_name(lname)
    pred = model.predict([np.array(fnamex),np.array(lnamex)])
    pred_class = np.argsort(pred[0])[::-1]
    return_item = []
    for x in np.argsort(pred[0])[::-1]:
        return_item.append((ethnic_keys[x],pred[0][x]))
    return return_item

In [49]:
predict_ethnicity('Hari','Cahyono')
#trans_name('Nikolaus')

[('arab', 0.69823658),
 ('hispanic', 0.057908442),
 ('french', 0.055705793),
 ('english', 0.048168365),
 ('israeli', 0.02471127),
 ('slav', 0.021401808),
 ('italian', 0.018654032),
 ('nordic', 0.017046874),
 ('indian', 0.016055189),
 ('turkish', 0.014107338),
 ('german', 0.007924147),
 ('chinese', 0.0059029222),
 ('romanian', 0.0056299986),
 ('african', 0.0029451125),
 ('japanese', 0.0026210765),
 ('baltic', 0.00069294788),
 ('dutch', 0.00057027361),
 ('thai', 0.00046238682),
 ('vietnamese', 0.00037071199),
 ('indonesian', 0.00027243383),
 ('greek', 0.00022111264),
 ('hungarian', 0.00022082729),
 ('korean', 0.000101094),
 ('caribbean', 3.0067344e-05),
 ('mongolian', 2.4259991e-05),
 ('polynesian', 1.4907315e-05)]

In [8]:
# embedd the structure vocabulary using text embedding and reduce the dimensionality

# convert the names into word structure vector
struct_dict_keys = list(struct_dict.keys())

def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

#data_source = full_name_struct.apply(lambda x: transform_structure(x))

In [9]:
def transform_labels(x):
    y = np.zeros(len(ethnic_keys))
    y[ethnic_keys.index(x)]=1
    return y

labels = np.array(list(map(lambda x: transform_labels(x),ethnic_series)))

In [26]:
# using tflearn make the graph creation simple
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split

# separate train and training set
trainX, testX, trainY, testY = train_test_split(data_source,[ethnic_keys.index(x) for x in ethnic_series],test_size = 0.2)

trainX = pad_sequences(trainX, maxlen=50,value=0.)
testX = pad_sequences(testX,maxlen=50,value=0.)
# Converting labels to binary vectors
trainY = to_categorical(trainY,nb_classes=len(ethnic_keys))
testY = to_categorical(testY,nb_classes=len(ethnic_keys))    

In [16]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import pickle

with open('train_test_fix.pickle', 'rb') as f:
    trainX,trainY,testX,testY,ethnic_keys,struct_dict_keys = pickle.load(f)
    #aha = pickle.load(f)

#with open('traintest-smote.pickle','rb') as f:
#    train_res,test_res = pickle.load(f)

with open('ethnic_keys.pickle','rb') as f:
    name_struct_keys,ethnic_keys = pickle.load(f)
        
embedding_vector_length = 1000
lstm_layer = 1000
max_sequence = 50

In [ ]:
# convert categorical to binary crossentropy
#trainY = np.array([np.where(x>0)[0][0] for x in trainY])
#testY = np.array([np.where(x>0)[0][0] for x in testY])

#test_res

In [2]:
model = Sequential()
model.add(Embedding(len(name_struct_keys)+1,embedding_vector_length,input_length=max_sequence))
model.add(Conv1D(filters=embedding_vector_length,kernel_size=3,padding='same',activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(LSTM(lstm_layer,dropout=0.8))
model.add(Bidirectional(LSTM(max_sequence*2,return_sequences=False),input_shape=(max_sequence,1)))
#model.add(TimeDistributed(keras.layers.Dense(len(ethnic_keys),activation='softmax')))
model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
#model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())
for x in range(10):
    model.fit(trainX,trainY,epochs=1,batch_size=1000)
    scores = model.evaluate(testX,testY,verbose=0)
    print("Accuracy: %.2f%%" %(scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 1000)          62696000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 1000)          3001000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               880800    
_________________________________________________________________
dense_1 (Dense)              (None, 23)                4623      
Total params: 66,582,423
Trainable params: 66,582,423
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
35653/35653 [==============================] - 778s - loss: 1.6879 - acc: 0.5265    
Accuracy: 75.88%
Epoch 1/1
35653/35653 [==============================] - 796s - loss: 0.5197 - acc: 0.8582    
Accuracy: 84.37%
Epoch 1/1
35653/35653 [==========

In [4]:
testX

array([[  112,  1331,  1332, ...,     0,     0,     0],
       [  251,   252,   232, ...,     0,     0,     0],
       [ 4633, 11731, 23123, ...,     0,     0,     0],
       ..., 
       [  696,   697,   698, ...,     0,     0,     0],
       [   25,    26,  5195, ...,     0,     0,     0],
       [ 8671,  8672,  6568, ...,     0,     0,     0]], dtype=int32)

In [12]:
from keras.models import model_from_json
from keras import backend as K

# compute the accuracy
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2
    #print(precision)

    # How many relevant items are selected?
    recall = c1 / c3
    #print(recall)

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def precision(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    return precision


def recall(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    recall = c1 / c3

    return recall

# load model
# load json and create model
json_file = open('model-keras-embed-bilstm-womaxpool.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#                               ,custom_objects= {'f1_score': f1_score})
loaded_model.load_weights("model-keras-embed-bilstm-womaxpool-10.h5")

loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy',f1_score,precision,recall])

In [18]:
scores = loaded_model.evaluate(testX,testY,verbose=0)

In [19]:
print('Accuracy: {}, F1: {}, Precision: {}, Recal: {}'.format(scores[1],scores[2],scores[3],scores[4]))

Accuracy: 0.8561812878216707, F1: 0.8601400889915978, Precision: 0.8757469038779302, Recal: 0.845523895023381


In [181]:
from tflearn.data_utils import to_categorical, pad_sequences

# transform prediction
# given name compute the prediction
def predict_ethnicity(name):
    # lower case the name
    name = name.lower()
    # transform space into underscore
    name = '_'+name.replace(' ','_')+'_'
    #transform the name into sequence structure
    ext_name = extract_structure(name)
    trans_name = transform_structure(ext_name)
    trans_name = pad_sequences([trans_name], maxlen=50,value=0.)
    pred = loaded_model.predict(trans_name)
    pred_class = np.argsort(pred[0])[::-1]
    return_item = []
    for x in np.argsort(pred[0])[::-1]:
        return_item.append((ethnic_keys[x],pred[0][x]))
    return return_item

name='helen lamothe'
ext_name = extract_structure(name)
#print(ext_name)
trans_name = transform_structure(ext_name)
#trans_name
pad_sequences([trans_name], maxlen=50,value=0.)
#extract_structure('Nikolaus Nova')
#transform_structure('Robert Nova')
ethnic_prob = predict_ethnicity('Filho  Elias Abdalla')
#ethnic_prob

In [132]:
test = loaded_model.predict(trainX[10].reshape(1,50))
test

array([[  1.22544670e-03,   2.45175033e-05,   5.54050894e-06,
          3.56604069e-05,   1.31730601e-04,   1.58663862e-03,
          9.95586514e-01,   3.65224201e-04,   4.18016425e-05,
          1.64734403e-04,   3.29397808e-05,   1.42851750e-05,
          5.77516516e-07,   3.13429664e-05,   1.98912196e-04,
          5.65968139e-06,   4.63458673e-06,   4.45792568e-04,
          5.28864875e-05,   2.15678101e-05,   8.53615438e-06,
          4.57901763e-07,   1.44505730e-05]], dtype=float32)

In [133]:
np.argsort(test)[0][::-1]

array([ 6,  5,  0, 17,  7, 14,  9,  4, 18,  8,  3, 10, 13,  1, 19, 22, 11,
       20, 15,  2, 16, 12, 21])

In [134]:
np.where(trainY[10]==1)

(array([6]),)

In [151]:
name='_helen__lamothe_'
ext_name = extract_structure(name)
#print(ext_name)
trans_name = transform_structure(ext_name)
#trans_name
pad_sequences([trans_name], maxlen=50,value=0.)

array([[    0,     0,     0,  2304,  2305,     0,     0,     0,     0,
         4088, 15559,  2523, 15102,     0,     0,     0,    24,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]], dtype=int32)

In [191]:
predict_ethnicity('harry potter')

[('ITALIAN', 0.52735686),
 ('INDIAN', 0.30269179),
 ('KOREAN', 0.097788125),
 ('JAPANESE', 0.017470013),
 ('ROMANIAN', 0.015210837),
 ('TURKISH', 0.014220745),
 ('HISPANIC', 0.0092350421),
 ('GERMAN', 0.0046770978),
 ('BALTIC', 0.0032188322),
 ('ARAB', 0.0028918688),
 ('ISRAELI', 0.0022467086),
 ('GREEK', 0.0014537659),
 ('SLAV', 0.00046977124),
 ('NORDIC', 0.00024971511),
 ('HUNGARIAN', 0.0002173665),
 ('DUTCH', 0.00021363674),
 ('INDONESIAN', 0.00019762212),
 ('VIETNAMESE', 9.9613972e-05),
 ('AFRICAN', 3.4369114e-05),
 ('CHINESE', 3.3702971e-05),
 ('ENGLISH', 1.7328795e-05),
 ('FRENCH', 4.1550811e-06),
 ('THAI', 9.4588233e-07)]

In [175]:
ethnic_keys[15]

'ISRAELI'

In [71]:
ethnea_df['First']+ethnea_df['Last']+ethnea_df['Ethnea']

0                           _Elias_Abdalla__Filho_HISPANIC
1                                  _Jad__Bou_Abdallah_ARAB
2                                  _Ayman__Abdel_Aziz_ARAB
3                                  _Salma__Abdelmoula_ARAB
4                                     _Ibrahim__Abdou_ARAB
5                             _Hazem__Abou_El_Fettouh_ARAB
6                                     _Rola__Aboutaam_ARAB
7        _Aida_Alexandra__Alvim_de_Abreu_Silva_Rodrigue...
8             _Isabel__Cristina_Affonso_Scaletsky_HISPANIC
9                                _Tsiri__Agbenyega_AFRICAN
10                     _Jose__Maria_Aguado_Garcia_HISPANIC
11                      _Manuela__Aguilar_Guisado_HISPANIC
12                         _Arturo__Aguillon_Luna_HISPANIC
13                                   _Ali__Ahmadzadeh_ARAB
14                                    _Ahmed__Ibrahim_ARAB
15                                    _Ahmed__Letaief_ARAB
16                           _Mohammed__Shakeel_Ahmed_AR